# 해외 여행, 언제 갈래? [여행정보 크롤링]

# PART 1 환율

In [86]:
import requests
import telegram

# (1) 환율정보 함수 만들기 [def 함수]

In [113]:
def 환율(country):
    URL = 'https://finance.naver.com/marketindex/exchangeList.naver'
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.select('tbody > tr')
    final_result = []
    result1 = soup.select('.tit')
    result2 = soup.select('.sale')

    for result in results:
        #tbody > tr 검색한 것이 통화량 환율 등 한 행이니까 반복문을 돌리고 있음
        #한 행별로 tit sale을 검색한다.(select_one 사용)
        title = result.select_one('.tit').text.strip()
        sale = result.select_one('.sale').text.strip()

        temp = {
            '통화명': title.split(),
            '환율': sale
        }

        final_result.append(temp) #딕셔너리로 만들어라!!! 키는 타이틀 스플릿해서 앞에것만 가져오라, 밸류는 sale 넣어라 (split)
        # {미국 : 1393.00, 유럽연합 : 1389.87} #정리한 다음에 미국, 유럽연합으로 함수의 인자(파라미터)를 받아라
        
    return final_result[country] #실습자료 보기

    #함수 안에 있는 변수는 밖에서 못씀
    #지역변수 전역변수
    #리턴을 달아주는 이유는 함수 안에서 실행돼서 밖에서 못쓰니까
    #어떻게든 전달위해!!
    #문자열로 만드는 연습을 해라

In [88]:
환율
#환율 미국 검색시 그래프랑 & 특정 환율만 보이게!
split


[{'통화명': '미국 USD', '환율': '1,393.00'},
 {'통화명': '유럽연합 EUR', '환율': '1,389.87'},
 {'통화명': '일본 JPY (100엔)', '환율': '970.90'},
 {'통화명': '중국 CNY', '환율': '198.38'},
 {'통화명': '홍콩 HKD', '환율': '177.46'},
 {'통화명': '대만 TWD', '환율': '44.32'},
 {'통화명': '영국 GBP', '환율': '1,584.26'},
 {'통화명': '오만 OMR', '환율': '3,622.89'},
 {'통화명': '캐나다 CAD', '환율': '1,046.58'},
 {'통화명': '스위스 CHF', '환율': '1,442.10'},
 {'통화명': '스웨덴 SEK', '환율': '128.80'},
 {'통화명': '호주 AUD', '환율': '930.94'},
 {'통화명': '뉴질랜드 NZD', '환율': '829.11'},
 {'통화명': '체코 CZK', '환율': '56.77'},
 {'통화명': '칠레 CLP', '환율': '1.51'},
 {'통화명': '터키 TRY', '환율': '76.17'},
 {'통화명': '몽골 MNT', '환율': '0.43'},
 {'통화명': '이스라엘 ILS', '환율': '403.74'},
 {'통화명': '덴마크 DKK', '환율': '186.87'},
 {'통화명': '노르웨이 NOK', '환율': '135.41'},
 {'통화명': '사우디아라비아 SAR', '환율': '370.66'},
 {'통화명': '쿠웨이트 KWD', '환율': '4,508.09'},
 {'통화명': '바레인 BHD', '환율': '3,695.06'},
 {'통화명': '아랍에미리트 AED', '환율': '379.25'},
 {'통화명': '요르단 JOD', '환율': '1,964.74'},
 {'통화명': '이집트 EGP', '환율': '71.66'},
 {'통화명': '태국 THB', '환

##(시현)내가 가장 가고 싶은 나라 2개 추출해보기 (미국 & 프랑스)

In [111]:
# 환율[0]
# type(환율[0])
# #직접 키값을 정해서 
# 딕셔너리 자체로 보내면 변환하는 과정이 있을 것
# 아예 문자열로 만들어서 보내야 함

use_data = 환율()[0:3]

message = ''
for ud in use_data:
    fx = ud['통화명']
    sale = ud['환율']
    message += str(fx) + ' ' + sale + '\n'

    #메시지 보내는 부분 추가하기

# (2) 10년간 국제 시장 환율 변동 측정

In [20]:
import requests
from bs4 import BeautifulSoup

In [21]:
BASE_URL = 'https://finance.naver.com/marketindex/exchangeDetail.naver?marketindexCd'
parameters = '=FX_EURKRW#'

URL = BASE_URL + parameters
res = requests.get(URL)

res.status_code

200

In [22]:
soup = BeautifulSoup(res.text, 'html.parser')
result = soup.select_one('.flash_area > img') 
graph = result.attrs['src']
#이미지를 크롤링
#속성으로 접근

print(graph)

https://ssl.pstatic.net/imgfinance/chart/marketindex/area/month3/FX_EURKRW.png


In [23]:
import telegram

In [24]:
telegram_config = {}
#config 파일 읽기
with open('./telegram_config', 'r') as f:
    #한 줄씩 읽어오기
    configs = f.readlines()
    #한 줄씩 확인해서
    for config in configs:
        #줄바꿈기호 제거 (\n) 후 =로 문자열 분리
        #key, value로 언패킹 (2개 나올 것이 확실하기 때문에)
        key, value = config.rstrip().split('=')
        #config 딕셔너리에 키-값 추가
        telegram_config[key] = value


print(telegram_config)
token = telegram_config['token']
chatId = telegram_config['chatId']
lastId = telegram_config['lastId']

{'token': '5782216675:AAH9FsDNrxOtRcsj-2ux63aoeQ55tbCwl0Q', 'chatId': '5662031911', 'lastId': '307922339'}


In [25]:
bot = telegram.Bot(token)
updates = bot.get_updates()
updates

In [26]:
chat_id = updates[-1].message.chat.id
chat_id

5662031911

In [106]:
img_url = 'https://ssl.pstatic.net/imgfinance/chart/marketindex/area/month3/FX_EURKRW.png?sidcode=20220919'
chat_id = telegram_config['chatId']

bot.sendPhoto(chat_id=chatId, photo=img_url, caption="더 자세한 환율정보를 원하시면 아래 링크를 클릭하세요.", reply_markup={
    "inline_keyboard": [
                           [
                               { "text": "FX_EURKRW", "url": "https://ssl.pstatic.net/imgfinance/chart/marketindex/area/month3/FX_EURKRW.png?sidcode=20220919" }
                           ]
                       ]})

In [ ]:
split()
[0] = 환율
if [0]== 환율 이면
[1] 값을 가지고
특정 환율정보만 보내주는식으로
미국
{
    '미국': 'USD',
    '유럽':'EUC'
} 